# Aeson

## Outline

* What is JSON

* AESON package

* Example

* CBOR standard

In this lecture, we will discuss the implementation of JSON in Haskell. The structure and sections are listed above. 

First, we will touch on what JSON is and how it is defined. After that, we will have a look how Haskell implements this. 

Lastly, we will give some extracurricular info on CBOR, which is similar to JSON.

## What is JSON

JSON is an abbreviation of **J**ava**S**cript **O**bject **N**otation, and it is a file format that is designed to interchange information in a human-readable form. 

The origin of JSON lies in the JavaScript language, this is why the syntax it uses looks similar to that language. The representation of data and its syntax follow these 4 rules:

Rule | Description | Example |
--- | --- | --- | 
1 | Data is in key/value pairs | `"firstName" : "Bob"`
2 | Data is separated by commas | `"firstName" : "Bob", "lastName" : "Smith"` 
3 | Curly brackets `{}` group objects in an unordered set | `{"one" : 1, "two" : 2, "two" : 3}`
4 | Square brackets `[]` hold arrays in an ordered collection| `"numbers": ["one", "two", "three"]`

The key value is always a string in JSON, this while the values can be of the types

Type | Examples
--- | --- |
String | `"name" : "Alice"`
Number | `{"one" : 210, "two" : -210, "three" : 3.14, "four" : 1.0E+2}`
Boolean | `"Haskell is cool" : true`
Null | `"key" : null `
Object | `{"favorite food" : "pasta"}`
Array | `[1,2,3]`

JSON is handy when structured data needs to be made visible. It is often used in API's and for computer-human interactions.


### AESON
The library that can parse and encode Haskell types to the JSON standard is called Aeson [(1)](https://hackage.haskell.org/package/aeson). 

This library is named after Aeson, the father of Jason, in Greek mythology. Aeson is not installed by default, since it is an external library. 

For the current notebook, it can be installed via the execution of

In [ ]:
:!stack build aeson

Like in the above JSON standard, the Aeson library has the `Value` type that has following possible values:
```haskell
-- | A JSON value represented as a Haskell value.
data Value = Object Object
           | Array Array         --- In JSON []
           | String Text         --- In JSON ""
           | Number Scientific   --- In JSON 3.1415E+0
           | Bool Bool           --- In JSON true/false
           | Null                --- In JSON null

-- A JSON "object" (key/value map).
type Object = KeyMap Value

-- A map from JSON key type Key to v.
data KeyMap v 
```

More explicitly, we have that we can encode these to JSON via the function `encode`.
```haskell
encode :: ToJSON a => a -> ByteString
```

In [ ]:
{-# LANGUAGE OverloadedStrings #-}
import Data.Aeson

encode [1,3,4]
encode "Hello World"
encode 3.1415
encode False
encode Null

To convert these back, we can use the `decode` function. Notice that the type signature of this function is always to a `Maybe a`. 
```haskell
decode :: FromJSON a => ByteString -> Maybe a 
```
This is because decoding a `ByteString` that could be in the JSON format might fail. Decoding the above examples gives

In [ ]:
{-# LANGUAGE OverloadedStrings #-}
import Data.Aeson

decode "[1,3,4]" :: Maybe [Int]
decode "\"Hello World\"" :: Maybe String
decode "3.1415" :: Maybe Double
decode "false" :: Maybe Bool
decode "null" :: Maybe Value

The `null` string is decoded as a `Maybe Value` since the Null is not a native object in Haskell, it is defined only in the data type `Value` given above.

### Example
If we have the data type 

In [7]:
data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int}

Haskell can derive from the type structure a general conversion into the JSON format. 

Here the type constructors are replaced with the key values (as strings), followed by the information in the fields.

To use this automated way of encoding Haskell types into JSON, you need the following things

- Import `import GHC.Generics`
- Use the pragma `{-# LANGUAGE DeriveGeneric  #-}`
- Use the pragma `{-# LANGUAGE DeriveAnyClass #-}`
- Use the pragma `{-# LANGUAGE OverloadedStrings #-}`
- Derive the instances of `Generic` for your type
- Derive the instances of `ToJSON`  and `FromJSON` for your type

These things are necesairy for Haskell to be able to convert the user made data type to a JSON string and back.

If you want to write JSON strings to a file or read from a JSON file you will also need the **Data.ByteString.Lazy** module.

Trying to acomplish this operation with the **Data.ByteString** or **Data.ByteString.Char8** modules will give a compile error.

Below is an example how to write data from a user made type to a JSON bystestring and read from it.

We also show that we can write this data to a file and then parse it back to the user made type.

In [ ]:
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE OverloadedStrings #-}

import Data.Aeson (decode, encode, FromJSON, ToJSON)
import GHC.Generics (Generic)
import Data.Maybe (fromJust)
import Data.ByteString.Lazy as B

data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int} 
                       deriving (Generic, Show, ToJSON, FromJSON)
                       
johnDoe :: Person
johnDoe = Person { firstName = "John"
                 , lastName = "Doe"
                 , age = 31}

main :: IO ()
main = do
    -- Shows how the encoded bystring looks like
    let jsonString = encode johnDoe
    print jsonString

    -- Shows how to decode a JSON Bytestring to the Person type
    let tomHanks = decode "{\"age\":66,\"lastName\":\"Hanks\",\"firstName\":\"Tom\"}" :: Maybe Person
    print $ fromJust tomHanks
    
    -- Shows how to write and read from JSON files
    B.writeFile "./person.json" jsonString
    johnDoeJSON <- B.readFile "./person.json"
    let johnDoe = decode johnDoeJSON :: Maybe Person
    print johnDoe

main

We see that for the type `Person` we can automatically derive the type classes `Generic`, `ToJSON` and `FromJSON`. 

This is because the type `Int` and `[Char]` which it contains, have an instance of `Generic` and can also be translated to the JSON `Value` type.

You may noticed that the order of the keys in the JSON bytestring for the tomHanks variable is in reverse order.

Never the less the decoding still works as it matches the key names with the function names of the Person type.

### Creating ToJSON and FromJSON instances

It is also possible to have a user made type that has different function names as the keys in a JSON object and parse the object to the type.

For this to be possible a ToJSON instance has to be created for the user defined type that handles the conversion.

If you want to perform this action in the oposite direction a FromJSON instance has also to be created.

When we create the instances for ToJSON and FromJSON for a type we need to use the aeson operators `.:` and `.=`.
```haskell
(.:) :: FromJSON a => Object -> Key -> Parser a 
(.=) :: ToJSON v => Key -> v -> kv 
```

The `.:` operator retrieves the value associated with the given key of an Object and returns is as a Paser type variable. 

The result is empty if the key is not present or the value cannot be converted to the desired type. 

The `.=` operator takes a key and a value, and returns a key-value result which type has to have an instance of the KeyValue type class.
```haskell
class KeyValue kv where
    (.=) :: ToJSON v => Key -> v -> kv
    infixr 8 .=
```

Below is a code example that reads the person.json file and decodes it to the type PersonNew which is a modified version of Person.

Then we change some data in it, and write it to a new file called personNew.json keeping the same key names as in the original file person.json.

In order for this code to work you will have to run the code from the previous cell that the person.json file gets created.

In [ ]:
import Data.Aeson (object, (.=), Value(Object), decode, encode, FromJSON (parseJSON), ToJSON)
import Data.Aeson.Types ((.:), ToJSON (toJSON))

data PersonNew = PersonNew { yourFirstName :: String
                           , yourLastName :: String
                           , yourAge :: Int} 
                             deriving Show

instance FromJSON PersonNew where
  parseJSON (Object p) =
      PersonNew <$> p .: "firstName"
                <*> p .: "lastName"
                <*> p .: "age"

instance ToJSON PersonNew where
  toJSON (PersonNew yourFirstName yourLastName yourAge) =
    object [ "firstName" .= yourFirstName
           , "lastName" .= yourLastName
           , "age" .= yourAge
           ]


main :: IO ()
main = do
    johnDoeJSON <- B.readFile "./person.json"
    let johnDoeNew = fromJust (decode johnDoeJSON :: Maybe PersonNew)
    print 
    
    let johnDoeNew' = johnDoeNew { yourAge = 32 }
        johnDoeNewJSON = encode johnDoeNew'
    B.writeFile "./personNew.json" johnDoeNewJSON

main

If we look now at the files person.json and personNew.json we see the key are the same and the value got updated.
- perosn.json
```json
{"age":31,"firstName":"John","lastName":"Doe"}
```

- perosnNew.json
```json
{"age":32,"firstName":"John","lastName":"Doe"}
```

The reason why this aproach is useful is in case a JSON file contains a key that is a reserved Haskell word.

Imagine a key is called `error` which can happen when you fetch some data from an web API. 

In such a case you need a different function name in your type and define a ToJSON and FromJSON instance for it.

### CBOR standard

The following information is extra and presented without code examples. 

In this lesson, we discussed the file format JSON that is designed to interchange information in a human-readable form. 

But in lesson 12 we learned that it is sometimes faster to use binary representation of things when we are processing them with a computer. 

The JSON standard still uses a lot of overhead to manage the data that it is trying to communicate. 

If we do not care about the human readability of the data, we can use the CBOR standard [(2)](https://hackage.haskell.org/package/CBOR) instead of the JSON standard. 

This stands for **C**oncise **B**inary **O**bject **R**epresentation and does the same as JSON but in a faster and more compressed way.